### **1.2 - ¿Como Airflow trabaja?**

#### **`Single Node Architecture `**

Empecemos con la arquitectura más sencilla, que es muy probable que empiece con la arquitectura de un solo nodo en esta arquitectura. Sólo tienes una máquina y los diferentes componentes del Web server, el scheduler, el metastore (o metadatabase) y el executor. Recuerde que el executor define cómo se ejecutan sus tareas, en qué sistema que se ejecutan.

Y como puedes ver, está el Executor, que es parte del Scheduler. Recuerda que el Executor no ejecuta una task, sino que define cómo y en qué sistema se ejecutan tus tasks. En este caso, sus tasks se ejecutan en procesos en una sola máquina (single machine).

<center><img src="https://i.postimg.cc/053gMQcC/a12.png"></center>

___

Recuerda que la metadatabase permite intercambiar datos entre los diferentes componentes de Airflow, y también hay un queue que permite ejecutar las tasks en el orden correcto. Siempre hay un queue, independientemente del executor que se utilice. Si ejecuta Airflow por primera vez, esta es la arquitectura con la que y tendrá el SequentialExecutor o el LocalExecutor, como veremos más adelante. vamos a ver más adelante en el curso.

<center><img src="https://i.postimg.cc/jS0Gc1Vw/a13.png"></center>

<center><img src="https://i.postimg.cc/pXnwDHSM/a14.png"></center>

___

#### **`¿Cómo funcionan estos componentes? `**

Bueno, en primer lugar, el **`Web server`** obtiene metadatos de la Metadatabase (Metastore) de Airflow, con el fin de mostrar la información correspondiente a sus DAGs, sus task instances o sus usuarios en la interfaz de usuario.

<center><img src="https://i.postimg.cc/6prFmhwH/a15.png"></center>

___

#### **`Apuntes del curso de Airflow Fundamentals`**

Luego está el **`Scheduler`**. Si una tarea está lista para ser programada (scheduled), el Scheduler cambiará el estado de esa tarea en el Metastore, entonces se crea un task instance object y ese task instance object es enviado desde el Scheduler al Executor y más específicamente a la Queue del Executor. Y ahí es donde la tarea estará lista para ser recuperada por un worker y así ser ejecutada.

Por último, pero no por ello menos importante, el **`Executor`** interactúa también con el metastore para actualizar las task stages. Tan pronto como la tarea se realiza el estado de esa tarea es cambiado por el Executor y no por el Scheduler.

De este esquema, se puede ver que el web server o el scheduler y el executor, todos ellos interactúan con la Metadatabase. Sin embargo, el web server no interactúa directamente con el Scheduler y el Executor.

Por lo tanto, esta es la arquitectura por defecto con la que terminará en cuanto configure y ejecute Airflow por primera vez.

<center><img src="https://i.postimg.cc/R0v25nZv/a16.png"></center>

___

#### **`Continuación Apuntes del curso de Udemy Introducción a Airflow`**

A continuación, el Scheduler interactúa con la Metadatabase y el executor para desencadenar (trigger) sus DAG's, para desencadenar (trigger) sus tasks. 

Finalmente, el executor interactúa también con la meta database para actualizar las tasks que acaban de ser completadas.  

Una cosa que hay que recordar es que este Scheduler, Executor y el Web server interactúan juntos gracias a la meta database de Airflow. Esto es lo primero que hay que recordar.

La segunda es que el Executor tiene una Queue interna y esta Queue es realmente parte del Executor si usas el Executor local, por ejemplo, y así es como tus tasks se ejecutan en el orden correcto porque hay una Queue en el Executor, **`recuerda que por defecto obtienes el orden de ejecución secuencial para ejecutar tus tasks una tras otra`**. Y si quieres empezar a configurar Airflow, puedes usar el Local Executor donde tus tasks se ejecutan en subprocesos con ambos Executors hay una Queue en él y así es como tus tasks se ejecutan en orden.

OK, así que esa arquitectura es muy bonita, pero no es suficiente.

<center><img src="https://i.postimg.cc/d0npqthr/a17.png"></center>

___

#### **`Multi Nodes Architecture (Celery) `**

Si quieres empezar a escalar cada vez que quieras ejecutar tantas tareas como quieras. En ese caso tendrás que pasar a otra arquitectura, que es la arquitectura multi nodos. Y más concretamente, normalmente esto se hace con Celery. Si no sabes lo que es Celery, te aconsejo encarecidamente que le eches un vistazo. Pero básicamente es una forma de procesar tus tasks en múltiples máquinas diferentes. Así que vamos a ver cómo será la arquitectura. En primer lugar, tendrás un nodo (la máquina) con diferentes componentes de airflow, como la Web server, el Scheduler y de nuevo, el Executor. Pero esta vez tendrás una segunda máquina donde normalmente pondrás la Metadatabase de Airflow así como la Queue. Esta vez la Queue es externa al Executor. En efecto, con el Executor Celery, no vas a tener una Queue interna, lo que significa que para configurar y ejecutar la queue donde las tareas serán empujadas (pushed) y extraídas (pulled), necesitarás un conjunto de herramientas como rabbitMQ o Redis, que se usarán para repartir y ejecutar tus tasks entre varias máquinas. Y esas máquinas son las siguientes:

El Worker número uno, dos y tres. En cada nodo worker (en cada máquina) tendrás otro componente de airflow llamado Airflow Worker, donde nuestras tasks serán ejecutadas.

<center><img src="https://i.postimg.cc/mD5nHXNr/a18.png"></center>

En resumen, tienes dos nodos. El primero contiene algunos componentes de airflow, como el Web server, el scheduler y el executor. Usted tiene un solo nodo de nuevo para los componentes de airflow, la Metadatabase y la Queue. Esta Queue es externa, sit es otra para apoyarse como RabbitMQ o Redis. Esta Queue se utilizará para repartir las tasks entre múltiples máquinas y entre múltiples nodos Worker. En cada nodo Worker tendrás otro componente de Airflow, un Airflow Worker, donde se ejecutarán las tasks.

Así que si usted echa un vistazo a las interacciones de nuevo, usted tiene el web server obtiene algunos datos de la meta database. A continuación, el Scheduler interactúa con el Executor y la meta database como antes. Una vez que la tarea está lista para ser programada (scheduled), esa tarea es enviada al Executor y el Executor envía la tarea a la queue (a la queue externa). Una vez que la tarea está en la queue, esa tarea está lista para ser extraída y ejecutada por uno de los workers y más específicamente por los Airflow workers. Así que cada uno de esos Workers obtendrá las tasks de la Queue desde RabbitMQ o Redis. Así es como funciona. Así es como puedes ejecutar múltiples tasks en diferentes máquinas.

Recuerda esto.

**`A continuación, al igual que con la arquitectura de nodo único, todavía el Scheduler y el Web Server intercambian datos con la metadatabase y el Executor envía las tareas a ejecutar a la Queue para ejecutarlas en el orden correcto, y finalmente, los Workers, los Airflow Workers sacan (pull) las tareas de la Queue y las ejecutan. Con esta arquitectura, si necesitas más recursos para ejecutar más tareas, sólo tienes que añadir un nuevo Airflow Worker en una nueva máquina`**. También tenga en cuenta que usted debe tener al menos dos Schedulers, así como dos Web Servers, tal vez un Load balancer delante de sus Web Servers para hacer frente al número de solicitudes en la interfaz de usuario de Airflow, así como PGBouncer para hacer frente al número de conexiones que se harán a su meta database.

<center><img src="https://i.postimg.cc/90wntdJY/a19.png"></center>

___

#### **`How it works (Task Lifecycle) `**

Primero creas un nuevo DAG, dag.py (un nuevo archivo Python) donde está definido tu data pipeline y pones ese archivo en la carpeta DAGs. 

<center><img src="https://i.postimg.cc/sXX81dW8/a20.png"></center>

A continuación, **`el Scheduler analiza esta carpeta DAGs cada cinco minutos por defecto para detectar nuevos DAGs`**. Por lo tanto, es posible que tenga que esperar hasta cinco minutos antes de obtener su DAG en la Airflow UI. 

**Parses** = _analizar_

<center><img src="https://i.postimg.cc/HjGPSrND/a21.png"></center>

**`A continuación, cada vez que aplique una modificación a ese DAG, es posible que tenga que esperar hasta 30 segundos antes de que aparezca la modificación. Por lo tanto, tenga en cuenta que cuando añade un nuevo DAG, el Scheduler analiza cada cinco minutos para los nuevos DAGs y para los DAGs existentes, el Scheduler analiza para las modificaciones cada 30 segundos`**. A continuación, el Scheduler ejecuta el DAG, y para ello, crea un objeto DagRun con el estado Running. 

<center><img src="https://i.postimg.cc/SQXStHPD/a22.png"></center>

Luego toma la primera tarea a ejecutar y esa tarea se convierte en un objeto instancia de tarea (task instance object). El objeto instancia de tarea (task instance object) tiene el estado None y luego Scheduled. 

<center><img src="https://i.postimg.cc/qB9JfjV0/a23.png"></center>

Después, el Scheduler envía el objeto de instancia de tarea (task instance object) a la Queue del Executor. 

<center><img src="https://i.postimg.cc/8PM1TMNv/a24.png"></center>

Ahora el estado de la tarea es Queue y el Executor crea un subproceso para ejecutar la tarea, y ahora el objeto de instancia de tarea (task instance object) tiene el estado Running. 

<center><img src="https://i.postimg.cc/W1Vszz1H/a25.png"></center>

<center><img src="https://i.postimg.cc/L8V21Sq6/a26.png"></center>

Una vez que la tarea se realiza, el estado de la tarea es Éxito (Success) o Fracaso (Failed). Depende. 

<center><img src="https://i.postimg.cc/VNL19hM9/a27.png"></center>

<center><img src="https://i.postimg.cc/Gp7RXjJW/a28.png"></center>

Y el Scheduler comprueba, si no hay tareas para ejecutar. Si el DAG está hecho en ese caso, el DAG Run tiene el estado Success.

<center><img src="https://i.postimg.cc/0QzsFJVb/a29.png"></center>

<center><img src="https://i.postimg.cc/wvnppTrF/a30.png"></center>

Y básicamente se puede actualizar la Airflow UI para comprobar los estados tanto de la ejecución del DAG como de las instancias de tareas de ese DAG Run. Así que ten en cuenta que cuando ejecutas un DAG, ese DAG tiene un DAG Run con el estado Queued, entonces, Success o Failed. Luego, cuando una tarea se ejecuta esa tarea se convierte en un objeto de instancia de tarea con primero, el estado None, luego Scheduled, luego Queued y luego Success o Failed. Hay otros estados, pero estos son los más importantes.

<center><img src="https://i.postimg.cc/zvW1XYZN/a31.png"></center>

___

#### **`Otra explicación`**

Ahora vamos a ampliar un poco para entender exactamente lo que sucede cuando se dispara una tarea (task is triggered), cuando se dispara un DAG.

Así que primero tienes los diferentes componentes de Airflow en un nodo: el servidor web, la meta database, el scheduler, el executor y por último, pero no menos importante, los Folder DAGs, donde estarán tus DAGs. 

Imaginemos que creas un DAG y lo depositas en esa Folder. Llamémoslo dag.py. Una vez que tu DAG esté en la Folder DAGs, tanto el scheduler como el web server analizarán tu DAG. El web server analiza los DAGs cada 30 segundos por defecto, mientras que el Scheduler analiza los nuevos DAGs y los nuevos archivos cada 5 minutos por defecto.

<center><img src="https://i.postimg.cc/kMtCyKVH/a32.png"></center>

Una vez analizado, el scheduler verificará si el DAG está listo para ser disparado (triggered), si es así el objeto DagRun es creado por el scheduler. **`Un objeto DagRun no es más que una instancia de su DAG que se ejecuta en un momento determinado`**. Y ese tipo de objeto se almacena en la base de datos de airflow con el estado "**`running`**".

<center><img src="https://i.postimg.cc/nc4tyxGY/a33.png"></center>

Al principio, las tareas de un objeto DagRun no tienen estado. Entonces, tan pronto como una tarea está lista para ser disparada (triggered) en su DAG, en ese caso, el scheduler crea un task instance object correspondiente a su task con el estado use "**`schedule`**" en la meta database de Airflow.

<center><img src="https://i.postimg.cc/tCQjvJW4/a34.png"></center>

A continuación, el Scheduler envía la tarea al Executor donde esta vez la tarea tiene el estado "**`queued`**".

<center><img src="https://i.postimg.cc/YCtwkPXc/a35.png"></center>

Y una vez que la tarea tiene el estado de "Queued" el executor está listo para tomar y ejecutar la task en un worker, esta vez el objeto TaskInstance tiene el estado "**`running`**" y el executor actualiza el estado de la task en la meta database de Airflow.

Y tan pronto como la tarea se realiza, el estado de esa tarea es "success" si todo ha funcionado como se esperaba y el estado del objeto TaskInstance es modificado por el Executor también a "**`success`**" en el Metastore.

<center><img src="https://i.postimg.cc/7L68J28m/a36.png"></center>

Ahora se han completado todas las tareas. El Scheduler comprueba si el trabajo está hecho, si no hay más tareas para programar (schedule), si no, el objeto DagRun se marca con el estado "success" si no hay ningún fallo en sus tareas y el web server actualiza la UI o en realidad, cada vez que actualice la página, verá si sus tareas están hechas o no.

<center><img src="https://i.postimg.cc/W1Sc47Kd/a37.png"></center>

<center><img src="https://i.postimg.cc/xd4rv19Y/a38.png"></center>